In [1]:
using JuMP
using Gurobi
# @testset "Lazy cut" begin
        model = Model(() -> Gurobi.Optimizer(Cuts=0, Presolve=0, Heuristics=0, LazyConstraints=1)) 
        #Gurobi.Optimizer() #OutputFlag=0, Cuts=0, Presolve=0, Heuristics=0, LazyConstraints=1)
#         MOI.Utilities.loadfromstring!(m,"""
#             variables: x, y
#             maxobjective: y
#             c1: x in Integer()
#             c2: y in Integer()
#             c3: x in Interval(0.0, 2.0)
#             c4: y in Interval(0.0, 2.0)
#         """)
#         x = MOI.get(m, MOI.VariableIndex, "x")
#         y = MOI.get(m, MOI.VariableIndex, "y")
        @variable(model,  0<= x <=2, Int)
        
        @variable(model, 0<= y <=2, Int)
        @objective(model, Max,y)
        # We now define our callback function that takes two arguments:
        #   (1) the callback handle; and
        #   (2) the location from where the callback was called.
        # Note that we can access m, x, and y because this function is defined
        # inside the same scope
        cb_calls = Int32[]
        function callback_function(cb_data::Gurobi.CallbackData, cb_where::Int32)
            push!(cb_calls, cb_where)
            if cb_where == Gurobi.CB_MIPSOL
                #Seems like MIPSOL indicates incumbent --> int sol?
                Gurobi.loadcbsolution!(m, cb_data, cb_where)
                x_val = MOI.get(m, MOI.VariablePrimal(), x)
                y_val = MOI.get(m, MOI.VariablePrimal(), y)
                # We have two constraints, one cutting off the top
                # left corner and one cutting off the top right corner, e.g.
                # (0,2) +---+---+ (2,2)
                #       |xx/ \xx|
                #       |x/   \x|
                #       |/     \|
                # (0,1) +       + (2,1)
                #       |       |
                # (0,0) +---+---+ (2,0)
                TOL = 1e-6  # Allow for some impreciseness in the solution
                if y_val - x_val > 1 + TOL
                    Gurobi.cblazy!(cb_data, m,
                        MOI.ScalarAffineFunction{Float64}(
                            MOI.ScalarAffineTerm.([-1.0, 1.0], [x, y]),
                            0.0
                        ),
                        MOI.LessThan{Float64}(1.0)
                    )
                elseif y_val + x_val > 3 + TOL
                    Gurobi.cblazy!(cb_data, m,
                        MOI.ScalarAffineFunction{Float64}(
                            MOI.ScalarAffineTerm.([1.0, 1.0], [x, y]),
                            0.0
                        ),
                        MOI.LessThan{Float64}(3.0)
                    )
                end
            end
        end
         MOI.set(model, MOI.LazyConstraintCallback(), my_callback_function)
         optimize!(model)

#         MOI.set(m, Gurobi.CallbackFunction(), callback_function)
#         MOI.optimize!(m)

#         @test MOI.get(m, MOI.VariablePrimal(), x) == 1
#         @test MOI.get(m, MOI.VariablePrimal(), y) == 2

#         @test length(cb_calls) > 0
#         @test Gurobi.CB_MESSAGE in cb_calls
#         @test Gurobi.CB_PRESOLVE in cb_calls
#         @test Gurobi.CB_MIPSOL in cb_calls
#     end

┌ Info: Recompiling stale cache file C:\Users\din\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\din\.julia\compiled\v1.0\Gurobi\do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1190
┌ Warning: Passing optimizer attributes as keyword arguments to
│ Gurobi.Optimizer is deprecated. Use
│     MOI.set(model, MOI.RawParameter("key"), value)
│ or
│     JuMP.set_optimizer_attribute(model, "key", value)
│ instead.
└ @ Gurobi C:\Users\din\.julia\packages\Gurobi\BAtIN\src\MOI_wrapper\MOI_wrapper.jl:273


UndefVarError: [91mUndefVarError: my_callback_function not defined[39m

In [2]:
using Pkg
Pkg.status()


    Status `C:\Users\din\.julia\environments\v1.0\Project.toml`
  [6cde8614] BlockDecomposition v1.4.0
  [336ed68f] CSV v0.8.5
  [88b4ec78] Coluna v0.3.10 [`C:\Users\din\.julia\dev\Coluna`]
  [861a8166] Combinatorics v1.0.2
  [a81c6b42] Compose v0.9.2
  [8f4d0f93] Conda v1.5.2
  [a93c6f00] DataFrames v1.2.2
  [31c24e10] Distributions v0.25.16
  [5789e2e9] FileIO v1.11.1
  [a2cc645c] GraphPlot v0.3.1
  [2e9cd046] Gurobi v0.9.14
  [7073ff75] IJulia v1.23.2
  [4076af6c] JuMP v0.21.10
  [86859df6] KnapsackLib v0.4.1 #main (https://github.com/rafaelmartinelli/KnapsackLib.jl)
  [093fc24a] LightGraphs v1.3.5
  [b8f27783] MathOptInterface v0.9.22
  [442fdcdd] Measures v0.3.1
  [eebad327] PkgVersion v0.1.1
  [91a5bcdd] Plots v1.0.14
  [f27b6e38] Polynomials v2.0.14
  [d330b81b] PyPlot v2.10.0
  [1a8c2f83] Query v0.12.2
  [47aef6b3] SimpleWeightedGraphs v1.1.1
  [a759f4b9] TimerOutputs v0.5.12
  [ade2ca70] Dates 
  [de0858da] Printf 


In [5]:
using JuMP, Gurobi, Test
model = direct_model(Gurobi.Optimizer())
@variable(model, 0 <= x <= 2.5, Int)
@variable(model, 0 <= y <= 2.5, Int)
@objective(model, Max, y)
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    # You can reference variables outside the function as normal
    push!(cb_calls, cb_where)
    # You can select where the callback is run
    if cb_where !=  Gurobi.CB_MIPSOL && cb_where !=  Gurobi.CB_MIPNODE #GRB_CB_MIPNODE
        return
    end
    # You can query a callback attribute using GRBcbget
    if cb_where == Gurobi.CB_MIPNODE # GRB_CB_MIPNODE
        resultP = Ref{Cint}()
        GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
        if resultP[] != GRB_OPTIMAL
            return  # Solution is something other than optimal.
        end
    end
    # Before querying `callback_value`, you must call:
    Gurobi.loadcbsolution!(model, cb_data, cb_where)
    x_val = MOI.get(m, MOI.VariablePrimal(), x)
    y_val = MOI.get(m, MOI.VariablePrimal(), y)
#     Gurobi.load_callback_variable_primal(cb_data, cb_where)
#     x_val = callback_value(cb_data, x)
#     y_val = callback_value(cb_data, y)
    # You can submit solver-independent MathOptInterface attributes such as
    # lazy constraints, user-cuts, and heuristic solutions.
    if y_val - x_val > 1 + 1e-6
        con = @build_constraint(y - x <= 1)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    elseif y_val + x_val > 3 + 1e-6
        con = @build_constraint(y + x <= 3)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
    if rand() < 0.1
        # You can terminate the callback as follows:
        GRBterminate(backend(model))
    end
    return
end
# You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
optimize!(model)
@test termination_status(model) == MOI.OPTIMAL
@test primal_status(model) == MOI.FEASIBLE_POINT
@test value(x) == 1
@test value(y) == 2

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)


UndefVarError: [91mUndefVarError: CB_MIPSOL not defined[39m

In [30]:
using MathOptInterface
function example_lazy_constraint()
    gurobi_env = Gurobi.Env()
    setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0,LazyConstraints=1)
    model =  Model(() -> Gurobi.Optimizer(gurobi_env))
    @variable(model, 0 <= x <= 2.5, Int)
    @variable(model, 0 <= y <= 2.5, Int)
    @objective(model, Max, y)
    lazy_called = false
    function my_callback_function(cb_data)
        lazy_called = true
        x_val = callback_value(cb_data, x)
        y_val = callback_value(cb_data, y)
        println("Called from (x, y) = ($x_val, $y_val)")
        status = callback_node_status(cb_data, model)
        if status == MOI.CALLBACK_NODE_STATUS_FRACTIONAL
            println(" - Solution is integer infeasible!")
        elseif status == MOI.CALLBACK_NODE_STATUS_INTEGER
            println(" - Solution is integer feasible!")
        else
            @assert status == MOI.CALLBACK_NODE_STATUS_UNKNOWN
            println(" - I don't know if the solution is integer feasible :(")
        end
        if y_val - x_val > 1 + 1e-6
            con = @build_constraint(y - x <= 1)
            println("Adding $(con)")
            MOI.submit(model, MOI.LazyConstraint(cb_data), con)
        elseif y_val + x_val > 3 + 1e-6
            con = @build_constraint(y - x <= 1)
            println("Adding $(con)")
            MOI.submit(model, MOI.LazyConstraint(cb_data), con)
        end
    end
    MOI.set(model, MOI.LazyConstraintCallback(), my_callback_function)
    optimize!(model)
    println("Optimal solution (x, y) = ($(value(x)), $(value(y)))")
    return
end

example_lazy_constraint()

Called from (x, y) = (-0.0, 2.0)


ArgumentError: [91mArgumentError: ModelLike of type Gurobi.Optimizer does not support accessing the attribute MathOptInterface.CallbackNodeStatus{CallbackData}(CallbackData(Ptr{Nothing} @0x0000000001971e80, Gurobi Model: [39m
[91m    type   : LP (MIP)[39m
[91m    sense  : maximize[39m
[91m    number of variables             = 2[39m
[91m    number of linear constraints    = 0[39m
[91m    number of quadratic constraints = 0[39m
[91m    number of sos constraints       = 0[39m
[91m    number of non-zero coeffs       = 0[39m
[91m    number of non-zero qp objective terms  = 0[39m
[91m    number of non-zero qp constraint terms = 0[39m
[91m))[39m

In [6]:
using JuMP, Gurobi, Test

model = direct_model(Gurobi.Optimizer())
@variable(model, 0 <= x <= 2.5, Int)
@variable(model, 0 <= y <= 2.5, Int)
@objective(model, Max, y)
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    # You can reference variables outside the function as normal
    push!(cb_calls, cb_where)
    # You can select where the callback is run
    if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
        return
    end
    # You can query a callback attribute using GRBcbget
    if cb_where == GRB_CB_MIPNODE
        resultP = Ref{Cint}()
        GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
        if resultP[] != GRB_OPTIMAL
            return  # Solution is something other than optimal.
        end
    end
    # Before querying `callback_value`, you must call:
    Gurobi.load_callback_variable_primal(cb_data, cb_where)
    x_val = callback_value(cb_data, x)
    y_val = callback_value(cb_data, y)
    # You can submit solver-independent MathOptInterface attributes such as
    # lazy constraints, user-cuts, and heuristic solutions.
    if y_val - x_val > 1 + 1e-6
        con = @build_constraint(y - x <= 1)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    elseif y_val + x_val > 3 + 1e-6
        con = @build_constraint(y + x <= 3)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
    if rand() < 0.1
        # You can terminate the callback as follows:
        GRBterminate(backend(model))
    end
    return
end
# You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
optimize!(model)
# @test termination_status(model) == MOI.OPTIMAL
# @test primal_status(model) == MOI.FEASIBLE_POINT
# @test value(x) == 1
# @test value(y) == 2


Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x434acaa1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e+00, 3e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.000000e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.0000000    2.00000  0.00%     -    0s

Cutting planes:
  Lazy constraints: 1

Explored 0 nodes (1 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 2 

In [9]:
using JuMP, Gurobi

model = direct_model(Gurobi.Optimizer())
@variable(model, x >= 0)
@constraint(model, c, 2*x >= 1)
@objective(model, Min, x)
MOI.set(model, Gurobi.ConstraintAttribute("Lazy"), c, 2)
optimize!(model)

# MOI.get(model, Gurobi.VariableAttribute("LB"), x)  # Returns 0.0
# MOI.get(model, Gurobi.ModelAttribute("NumConstrs")) # Returns 1

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1 rows, 1 columns and 1 nonzeros
Model fingerprint: 0x1c9f10d3
Coefficient statistics:
  Matrix range     [2e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  5.000000000e-01

User-callback calls 24, time in user-callback 0.00 sec


In [8]:
using Gurobi, Test, Random
const MOI = Gurobi.MOI
const GUROBI_ENV = Gurobi.Env()
function callback_simple_model()
    model = Gurobi.Optimizer(
        GUROBI_ENV,
#         OutputFlag = 0,
        Cuts = 0,
        Presolve = 0,
        Heuristics = 0,
    )
    MOI.Utilities.loadfromstring!(model, """
        variables: x, y
        maxobjective: y
        c1: x in Integer()
        c2: y in Integer()
        c3: x in Interval(0.0, 2.5)
        c4: y in Interval(0.0, 2.5)
    """)
    x = MOI.get(model, MOI.VariableIndex, "x")
    y = MOI.get(model, MOI.VariableIndex, "y")
    return model, x, y
end

model, x, y = callback_simple_model()
cb_calls = Int32[]
function callback_function(cb_data::Gurobi.CallbackData, cb_where::Int32)
    push!(cb_calls, cb_where)
    if cb_where == Gurobi.CB_MIPSOL
        Gurobi.cbget_mipsol_sol(model, cb_data, cb_where)
        x_val = MOI.get(model, MOI.CallbackVariablePrimal(cb_data), x)
        y_val = MOI.get(model, MOI.CallbackVariablePrimal(cb_data), y)
        if y_val - x_val > 1 + 1e-6
            MOI.submit(model, MOI.LazyConstraint(cb_data),
                MOI.ScalarAffineFunction{Float64}(
                    MOI.ScalarAffineTerm.([-1.0, 1.0], [x, y]),
                    0.0
                ),
                MOI.LessThan{Float64}(1.0)
            )
        elseif y_val + x_val > 3 + 1e-6
            MOI.submit(model, MOI.LazyConstraint(cb_data),
                MOI.ScalarAffineFunction{Float64}(
                    MOI.ScalarAffineTerm.([1.0, 1.0], [x, y]),
                    0.0
                ),
                MOI.LessThan{Float64}(3.0)
            )
        end
    end
end
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), callback_function)
MOI.optimize!(model)
# @test MOI.get(model, MOI.VariablePrimal(), x) == 1
# @test MOI.get(model, MOI.VariablePrimal(), y) == 2
# @test length(cb_calls) > 0
# @test Gurobi.CB_MESSAGE in cb_calls
# @test Gurobi.CB_PRESOLVE in cb_calls
# @test Gurobi.CB_MIPSOL in cb_calls

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e+00, 3e+00]
  RHS range        [0e+00, 0e+00]
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.000000e+00, 0 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.0000000    2.00000  0.00%     -    0s

Cutting planes:
  Lazy constraints: 1

Explored 0 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.000000000000e+00, best bound 2.000000000000e+00, gap 0.0000%


In [2]:
#Gurobi v0.9
using JuMP, Gurobi, Test

model = direct_model(Gurobi.Optimizer())
@variable(model, 0 <= x <= 2.5, Int)
@variable(model, 0 <= y <= 2.5, Int)
@objective(model, Max, y)
cb_calls = Cint[]
function my_callback_function(cb_data, cb_where::Cint)
    # You can reference variables outside the function as normal
    push!(cb_calls, cb_where)
    # You can select where the callback is run
    if cb_where != GRB_CB_MIPSOL && cb_where != GRB_CB_MIPNODE
        return
    end
    # You can query a callback attribute using GRBcbget
    if cb_where == GRB_CB_MIPNODE
        resultP = Ref{Cint}()
        GRBcbget(cb_data, cb_where, GRB_CB_MIPNODE_STATUS, resultP)
        if resultP[] != GRB_OPTIMAL
            return  # Solution is something other than optimal.
        end
    end
    # Before querying `callback_value`, you must call:
    Gurobi.load_callback_variable_primal(cb_data, cb_where)
    x_val = callback_value(cb_data, x)
    y_val = callback_value(cb_data, y)
    # You can submit solver-independent MathOptInterface attributes such as
    # lazy constraints, user-cuts, and heuristic solutions.
    if y_val - x_val > 1 + 1e-6
        con = @build_constraint(y - x <= 1)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    elseif y_val + x_val > 3 + 1e-6
        con = @build_constraint(y + x <= 3)
        MOI.submit(model, MOI.LazyConstraint(cb_data), con)
    end
#     if rand() < 0.1
#         # You can terminate the callback as follows:
#         GRBterminate(backend(model))
#     end
    return
end
# You _must_ set this parameter if using lazy constraints.
MOI.set(model, MOI.RawParameter("LazyConstraints"), 1)
MOI.set(model, Gurobi.CallbackFunction(), my_callback_function)
optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0x434acaa1
Variable types: 0 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [3e+00, 3e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 0 rows, 2 columns, 0 nonzeros
Variable types: 0 continuous, 2 integer (0 binary)

Root relaxation: objective 2.000000e+00, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       2.0000000    2.00000  0.00%     -    0s

Cutting planes:
  Lazy constraints: 1

Explored 0 nodes (1 simplex iterations) in 0.00 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 2 

In [14]:
# The input data is from page 139, Integer programming by Garfinkel and
# Nemhauser[[1]](#c1).

c1 = [-1; -4]
c2 = [-2; -3]

dim_x = length(c1)
dim_u = length(c2)

b = [-2; -3]

A1 = [
    1 -3
    -1 -3
]
A2 = [
    1 -2
    -1 -1
]

M = 1000;

# ## Implementation

using JuMP, Gurobi, Test
# import GLPK
# import Test

# !!! info
#     This tutorial uses the [MathOptInterface](@ref moi_documentation) API.
#     By default, JuMP exports the `MOI` symbol as an alias for the
#     MathOptInterface.jl package. We recommend making this more explicit in
#     your code by adding the following lines:
#     ```julia
#     import MathOptInterface
#     const MOI = MathOptInterface
#     ```

# Subproblem creation

function build_subproblem()
    sub_problem_model = Model(Gurobi.Optimizer)
    @variable(sub_problem_model, u[1:dim_u] >= 0)
    @constraint(
        sub_problem_model,
        constr_ref_subproblem[j = 1:size(A2, 2)],
        A2[:, j]' * u >= c2[j],
    )
    return (sub_problem_model, u)
end

# ### Master Problem Description

master_problem_model = Model(Gurobi.Optimizer)
@variable(master_problem_model, 0 <= x[1:dim_x] <= 1e6, Int)
@variable(master_problem_model, t <= 1e6)
@objective(master_problem_model, Max, t)
print(master_problem_model)

#-

(sub_problem_model, u) = build_subproblem();

# Track the calls to the callback

iter_num = 0

# Define lazy constraints

function benders_lazy_constraint_callback(cb_data)
    global iter_num
    iter_num += 1
    println("Iteration number = ", iter_num)

    x_current = callback_value.(Ref(cb_data), x)
    fm_current = callback_value(cb_data, t)

    c_sub = b - A1 * x_current
    @objective(sub_problem_model, Min, c1' * x_current + c_sub' * u)
    optimize!(sub_problem_model)

    print("\nThe current subproblem model is \n")
    print(sub_problem_model)

    t_status_sub = termination_status(sub_problem_model)
    p_status_sub = primal_status(sub_problem_model)
    println("p_status_sub ", p_status_sub)
    fs_x_current = objective_value(sub_problem_model)

    u_current = value.(u)

    γ = b' * u_current

    if p_status_sub == MOI.FEASIBLE_POINT && fs_x_current ≈ fm_current # we are done
        @info("No additional constraint from the subproblem")
    end

    if p_status_sub == MOI.FEASIBLE_POINT && fs_x_current < fm_current
        println(
            "\nThere is a suboptimal vertex, add the corresponding constraint",
        )
        cv = A1' * u_current - c1
        new_optimality_cons = @build_constraint(t + cv' * x <= γ)
        MOI.submit(
            master_problem_model,
            MOI.LazyConstraint(cb_data),
            new_optimality_cons,
        )
    end

    if t_status_sub == MOI.INFEASIBLE_OR_UNBOUNDED
        println(
            "\nThere is an  extreme ray, adding the corresponding constraint",
        )
        ce = A1' * u_current
        new_feasibility_cons = @build_constraint(dot(ce, x) <= γ)
        MOI.submit(
            master_problem_model,
            MOI.LazyConstraint(cb_data),
            new_feasibility_cons,
        )
    end
end

MOI.set(
    master_problem_model,
    MOI.LazyConstraintCallback(),
    benders_lazy_constraint_callback,
)

optimize!(master_problem_model)

t_status = termination_status(master_problem_model)
p_status = primal_status(master_problem_model)
println("p_status ", p_status)
if p_status == MOI.INFEASIBLE_POINT
    println("The problem is infeasible :-(")
end

if t_status == MOI.INFEASIBLE_OR_UNBOUNDED
    fm_current = M
    x_current = M * ones(dim_x)
end

if p_status == MOI.FEASIBLE_POINT
    fm_current = value(t)
    x_current = value.(x)
end

# Test.@test value(t) ≈ -4 #hide

println(
    "Status of the master problem is ",
    t_status,
    "\nwith fm_current = ",
    fm_current,
    "\nx_current = ",
    x_current,
)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0xd2f72427
Variable types: 1 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+06, 1e+06]
  RHS range        [0e+00, 0e+00]
Iteration number = 1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x857e2bca
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.

┌ Info: No additional constraint from the subproblem
└ @ Main In[14]:95
┌ Info: No additional constraint from the subproblem
└ @ Main In[14]:95


[0.0, 1.0]
